# exp035_ea
exp035の誤差分析

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve
pd.set_option('display.max_rows', 100)

In [2]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [3]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [4]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [6]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [7]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [8]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [9]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [10]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = get_ground_truth()
    return base_train, base_test, sample_sub, ground_truth

In [11]:
train_pred = pd.read_csv('../output/exp035/train_ro_rog_kf_sp0_pm_ps.csv')
sub = pd.read_csv('../output/exp035/exp035_submission.csv')

In [12]:
train, test, _, gt = get_data()

In [13]:
gt = gt.reset_index(drop=True)

In [15]:
tmp

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,phone,heightAboveWgs84EllipsoidM,x,y,z,...,zdiff,dist,xnew,ynew,znew,timeSinceFirstFixSeconds,hDop,vDop,speedMps,courseDegree
0,2020-05-14-US-MTV-1,pred,1273529463442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.50,-2.694608e+06,-4.296551e+06,3.854871e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-14-US-MTV-1,pred,1273529464442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.50,-2.694608e+06,-4.296551e+06,3.854871e+06,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-14-US-MTV-1,pred,1273529465442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.50,-2.694608e+06,-4.296551e+06,3.854871e+06,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-14-US-MTV-1,pred,1273529466442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.50,-2.694608e+06,-4.296551e+06,3.854871e+06,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-14-US-MTV-1,pred,1273529467442,37.423583,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.50,-2.694608e+06,-4.296552e+06,3.854870e+06,...,-0.380402,0.490277,-2.694608e+06,-4.296551e+06,3.854871e+06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131337,2020-05-29-US-MTV-2,gt,1274832059447,37.415923,-122.080712,NaN,37.54,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2120.45,0.95,0.0,0.0,303.1
131338,2020-05-29-US-MTV-2,gt,1274832060447,37.415923,-122.080712,NaN,37.54,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2121.45,1.40,0.0,0.0,303.1
131339,2020-05-29-US-MTV-2,gt,1274832061447,37.415923,-122.080712,NaN,37.54,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2122.45,1.60,0.0,0.0,303.1
131340,2020-05-29-US-MTV-2,gt,1274832062447,37.415923,-122.080712,NaN,37.54,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2123.45,1.75,0.0,0.0,303.1


In [19]:
tmp1 = train_pred.copy()
tmp2 = gt.copy()

tmp1['phoneName'] = 'pred'
tmp2['phoneName'] = 'gt'

tmp = pd.concat([tmp1, tmp2])

visualize_collection(tmp1, '2021-04-22-US-SJC-1_Pixel4')

KeyError: 'phoneName'